In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import pandas as pd

df = pd.read_csv('data/reddit_data.csv')
gold_data = pd.read_csv('data/full_dataset.csv')

/tmp/ipykernel_457991/3648894930.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/reddit_data.csv')


In [8]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1').to('cuda')

# Two lists of sentences


gold_sents = gold_data[gold_data['label'] == 'Violates']['text'].to_list()
# gold_sents = gold_data['0'].to_list()

#Compute embedding for both lists
gold_embed = model.encode(gold_sents, convert_to_tensor=True).to('cuda')

to_keep = []


In [9]:
df['text'].isna().value_counts()

False    28458925
True       553105
Name: text, dtype: int64

In [10]:

df = df[~df['text'].isna()].sample(n=3000000)

In [11]:
from tqdm.notebook import tqdm
to_keep = []

negatives = []

thres = 0.55
min_thres = 0.3

#Compute cosine-similarits
for i in tqdm(range(len(df))):
    try:
        text = model.encode(df['text'].iloc[i],  convert_to_tensor=True).to('cuda')

        cosine_scores = util.pytorch_cos_sim(text, gold_embed)
    except:
        continue
    #print(cosine_scores[0])

    if max(cosine_scores[0]) > thres:
        to_keep.append(i)
        if len(to_keep) >= 100000:
            break
    elif max(cosine_scores[0]) < min_thres:
        negatives.append(i)


  0%|          | 0/3000000 [00:00<?, ?it/s]

In [14]:
len(to_keep), len(negatives)

(100000, 11628)

In [23]:
df.iloc[negatives].text.str.len().describe()

count    11628.000000
mean       353.375989
std        340.312550
min          8.000000
25%        163.000000
50%        263.000000
75%        431.000000
max       9967.000000
Name: text, dtype: float64

In [25]:
import random
df.iloc[to_keep].to_csv('data/similar_reddit.csv')

In [24]:
df.iloc[negatives][(df.text.str.len() > 3)].dropna().sample(n=5000).to_csv('data/negative_for_dataset.csv')

/tmp/ipykernel_457991/3423515588.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.iloc[negatives][(df.text.str.len() > 3)].dropna().sample(n=5000).to_csv('data/negative_for_dataset.csv')


In [39]:


(df[df.index.isin(to_keep)]['subreddit'].value_counts()/df.iloc[range(217846)]['subreddit'].value_counts()).dropna()

AmItheAsshole            0.035730
AskMen                   0.041248
Bad_Cop_No_Donut         0.027778
EnoughLibertarianSpam    0.018182
Fuckthealtright          0.013477
Impeach_Trump            0.025000
NoStupidQuestions        0.038508
PoliticalHumor           0.020149
ShitLiberalsSay          0.031214
ShitPoliticsSays         0.030576
Showerthoughts           0.029432
dadjokes                 0.033793
esist                    0.024038
funny                    0.041057
legaladvice              0.032890
mildlyinteresting        0.050222
todayilearned            0.045959
worldpolitics            0.031766
Name: subreddit, dtype: float64